In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
import random

%matplotlib notebook

In [2]:
# current function for math solution, using a bouncing ball in a 0-1 square box
def get_next_step(position, velocity, delta):
    global elasticity, drag
    if position[1] < 0:
        position[1] = 0
        velocity[1] = -elasticity*velocity[1]
    if position[1] > 1:
        position[1] = 1
        velocity[1] = -elasticity*velocity[1]
    if position[0] < 0:
        position[0] = 0
        velocity[0] = -elasticity*velocity[0]
    if position[0] > 1:
        position[0] = 1
        velocity[0] = -elasticity*velocity[0]
    
    next_position = np.sum([position,velocity*delta], axis = 0)


    acceleration_x = -drag*velocity[0]**2
    acceleration_y = (-drag*velocity[1]**2) - 9.81


    next_velocity_x = velocity[0] + acceleration_x*delta
    next_velocity_y = velocity[1] + acceleration_y*delta

    next_velocity = np.array([next_velocity_x, next_velocity_y])
    
    #print(f'Position: {next_position}      Velcocity = {next_velocity}')

    return next_position, next_velocity

def get_starting_values():
    x0 = random.random()
    y0 = random.random()
    
    v0x = (0.5-random.random())*20
    v0y = (0.5-random.random())*20
    
    elasticity = 0.4 + random.random()*0.5
    drag = 0.2 + random.random()*0.8
    
    return x0, y0, v0x, v0y, elasticity, drag

def calculate_positions(time, delta, x0, y0, v0x, v0y, elasticity, drag):
    timesteps = np.arange(0, time, delta)
    
    positions = np.array([[x0,y0]])
    velocities = np.array([[v0x,v0y]])
    
    for index, step in enumerate(timesteps):
        
        newposition, newvelocity = get_next_step(positions[index], velocities[index], delta)

        positions = np.append(positions, [newposition], axis = 0)
        velocities = np.append(velocities, [newvelocity], axis = 0)
        
    return positions

def make_animation(positions, name,dpi=150):
    fig, ax = plt.subplots(figsize=(4,4))
    xdata, ydata = [], []
    ax.axis('equal')
    ax.axis('off')
    ax.set_position([0, 0, 1, 1])

    #draw a bounding box
    pad = 0.02
    ax.plot([0-pad, 1+pad, 1+pad,0-pad,0-pad], [0-pad,0-pad,1+pad,1+pad,0-pad], c = 'k')
    ball = Circle((x0,y0), radius = 0.02, color='k')
    ax.add_patch(ball)

    ln, = ax.plot([0,0],[0,0],color = 'k', lw=0.5)  

    def init():
        ax.set_xlim(-0.2, 1.2)
        ax.set_ylim(-0.2, 1.2)
        count = 0
        return ln,

    global tail
    tail = 10
    def update(frame):
        global points    
        if frame < tail:
            ln.set_data(positions[:frame+1,0], positions[:frame+1,1])
        else:
            ln.set_data(positions[frame+1-tail:frame,0], positions[frame+1-tail:frame,1])

        ball.center = positions[frame, 0], positions[frame, 1]
        return ln,

    ani = FuncAnimation(fig, update, frames=np.arange(len(positions)),
                        init_func=init, blit=True, interval=1/30)   #time/steps

    writergif = animation.PillowWriter(fps=30)
    ani.save(f'temp/{name}.gif',writer=writergif,dpi=dpi)
    plt.close()
    print(f'{name}.gif saved! \n')


In [3]:
%matplotlib inline

In [4]:
global time, delta

time = 3
speed = 0.5
delta = 1/30*speed

for i in range(10):
    x0, y0, v0x, v0y, elasticity, drag = get_starting_values()
    positions = calculate_positions(time, delta, x0, y0, v0x, v0y, elasticity, drag)
    make_animation(positions,f'{i+1}',100)


1.gif saved! 

2.gif saved! 

3.gif saved! 

4.gif saved! 

5.gif saved! 

6.gif saved! 

7.gif saved! 

8.gif saved! 

9.gif saved! 

10.gif saved! 



In [5]:
def get_starting_values():
    x0 = random.random()
    y0 = random.random()
    
    v0x = (0.5-random.random())*20
    v0y = (0.5-random.random())*20
    
    elasticity = 0.4 + random.random()*0.4
    drag = 0.2 + random.random()*0.8
    
    return x0, y0, v0x, v0y, elasticity, drag

def calculate_2_positions(starting_values, elastics=None, drags=None):
    global elasticity, drag, index
    timesteps = np.arange(0, time, delta)
    
    a = starting_values[0]
    b = starting_values[1]

    a_positions = np.array([[a[0],a[1]]])
    a_velocities = np.array([[a[2],a[3]]])
    
    b_positions = np.array([[b[0],b[1]]])
    b_velocities = np.array([[b[2],b[3]]])

    if not elastics:
        elasticity = a[4]
    else:
        elasticity = elastics
    if not drags:
        drag = a[5]
    else:
        drag = drags
    
    for index, step in enumerate(timesteps):
        a,b,av,bv = a_positions[index], b_positions[index], a_velocities[index], b_velocities[index]
        
        a_position, b_position, a_velocity, b_velocity = get_2_next_steps(a, b, av, bv, delta)

        a_positions = np.append(a_positions, [a_position], axis = 0)
        a_velocities = np.append(a_velocities, [a_velocity], axis = 0)
        
        b_positions = np.append(b_positions, [b_position], axis = 0)
        b_velocities = np.append(b_velocities, [b_velocity], axis = 0)
        
    return a_positions, b_positions

def get_2_next_steps(a_position, b_position, a_velocity, b_velocity, delta):
    
    distance = np.sqrt(((a_position[0]-b_position[0])**2) + ((a_position[1]-b_position[1])**2))

    if distance > 2*radius:        
        if a_position[1] < radius:
            a_position[1] = radius
            a_velocity[1] = -elasticity*a_velocity[1]
        if a_position[1] > 1-radius:
            a_position[1] = 1-radius
            a_velocity[1] = -elasticity*a_velocity[1]
        if a_position[0] < radius:
            a_position[0] = radius
            a_velocity[0] = -elasticity*a_velocity[0]
        if a_position[0] > 1-radius:
            a_position[0] = 1-radius
            a_velocity[0] = -elasticity*a_velocity[0]
            
        if b_position[1] < radius:
            b_position[1] = radius
            b_velocity[1] = -elasticity*b_velocity[1]
        if b_position[1] > 1-radius:
            b_position[1] = 1-radius
            b_velocity[1] = -elasticity*b_velocity[1]
        if b_position[0] < radius:
            b_position[0] = radius
            b_velocity[0] = -elasticity*b_velocity[0]
        if b_position[0] > 1-radius:
            b_position[0] = 1-radius
            b_velocity[0] = -elasticity*b_velocity[0]
    else:
        collison_elastics = 1.1
        ax,ay = a_velocity.copy()
        a_velocity[0] = b_velocity[0]*collison_elastics
        a_velocity[1] = b_velocity[1]*collison_elastics
        
        b_velocity[0] = ax*collison_elastics
        b_velocity[1] = ay*collison_elastics
    
    next_a_position = np.sum([a_position, a_velocity*delta], axis = 0)
    next_b_position = np.sum([b_position, b_velocity*delta], axis = 0)


    
    acceleration_a_x = -drag*a_velocity[0]**2
    acceleration_a_y = (-drag*a_velocity[1]**2) - 9.81

    acceleration_b_x = -drag*b_velocity[0]**2
    acceleration_b_y = (-drag*b_velocity[1]**2) - 9.81

    next_velocity_a_x = a_velocity[0] + acceleration_a_x*delta
    next_velocity_a_y = a_velocity[1] + acceleration_a_y*delta

    next_velocity_b_x = b_velocity[0] + acceleration_b_x*delta
    next_velocity_b_y = b_velocity[1] + acceleration_b_y*delta    

    next_a_velocity = np.array([next_velocity_a_x, next_velocity_a_y])
    next_b_velocity = np.array([next_velocity_b_x, next_velocity_b_y])

    #print(f'Position: {next_position}      Velcocity = {next_velocity}')

    return next_a_position, next_b_position, next_a_velocity,next_b_velocity



def make_2_animation(a_positions, b_positions, name, dpi=150,save=False):
    fig, ax = plt.subplots(figsize=(4,4))
    xdata, ydata = [], []
    ax.axis('equal')
    ax.axis('off')
    ax.set_position([0, 0, 1, 1])

    #draw a bounding box
    pad = 0.02
    ax.plot([0-pad, 1+pad, 1+pad,0-pad,0-pad], [0-pad,0-pad,1+pad,1+pad,0-pad], c = 'k')
    
    a_ball = Circle((a_positions[0][0],a_positions[0][1]), radius = radius, color='b')
    ax.add_patch(a_ball)
    b_ball = Circle((b_positions[0][0],b_positions[0][1]), radius = radius, color='r')
    ax.add_patch(b_ball)
    

    a_ln, = ax.plot([0,0],[0,0],color = 'b', lw=0.5)  
    b_ln, = ax.plot([0,0],[0,0],color = 'r', lw=0.5)  

    
    def init():
        ax.set_xlim(-0.2, 1.2)
        ax.set_ylim(-0.2, 1.2)
        return a_ln, b_ln

    global tail
    tail = 12
    def update(frame):
        global points    
        if frame < tail:
            a_ln.set_data(a_positions[:frame+1,0], a_positions[:frame+1,1])
            b_ln.set_data(b_positions[:frame+1,0], b_positions[:frame+1,1])
        else:
            a_ln.set_data(a_positions[frame+1-tail:frame,0], a_positions[frame+1-tail:frame,1])
            b_ln.set_data(b_positions[frame+1-tail:frame,0], b_positions[frame+1-tail:frame,1])

        a_ball.center = a_positions[frame, 0], a_positions[frame, 1]
        b_ball.center = b_positions[frame, 0], b_positions[frame, 1]
        
        return a_ln, b_ln

    ani = FuncAnimation(fig, update, frames=np.arange(len(a_positions)),
                        init_func=init, blit=True, interval=1/30)   #time/steps

    if save:
        writergif = animation.PillowWriter(fps=30)
        ani.save(f'temp/{name}.gif',writer=writergif,dpi=dpi)
        plt.close()
        print(f'{name}.gif saved! \n')
    if not save:
        plt.show();

In [6]:
#now lets add 2 balls? and collision dynamics?
global time, delta, radius
%matplotlib inline

time = 3
speed = 0.3
delta = (1/30)*speed

radius = 0.02
elasticity = 0.8
drag = 0.6

for i in range(5):
    starting_vals = [get_starting_values(), get_starting_values()]

    a_positions, b_positions = calculate_2_positions(starting_vals, elasticity, drag)

    make_2_animation(a_positions, b_positions,f'Collisions - {i+1}', 100, save=True)

Collisions - 1.gif saved! 

Collisions - 2.gif saved! 

Collisions - 3.gif saved! 

Collisions - 4.gif saved! 

Collisions - 5.gif saved! 



In [ ]:
    next_a_position[0] = next_a_position[0] - np.cos(angle)*radius
    next_a_position[1] = next_a_position[1] - np.sin(angle)*radius
    next_b_position[0] = next_b_position[0] - np.cos(angle)*radius
    next_b_position[1] = next_b_position[1] - np.sin(angle)*radius


    delta_x = next_a_position[0] - next_b_position[0]
    delta_y = next_a_position[1] - next_b_position[1]
    center = [(next_a_position[0]+next_b_position[0])/2, (next_a_position[1]+next_b_position[1])/2]
    angle = np.arctan(delta_y/delta_x)
    idealx = np.cos(angle)*radius
    ideay = np.sin(angle)*radius

    adx = center[0] - next_a_position[0]

    next_a_position[0] = center[0]
    next_a_position[1] = center[1] + np.sin(angle)*radius
    next_b_position[0] = next_b_position[0] - np.cos(angle)*radius
    next_b_position[1] = next_b_position[1] - np.sin(angle)*radius




global new_distance
new_distance = np.sqrt(((next_a_position[0]-next_b_position[0])**2) + ((next_a_position[1]-next_b_position[1])**2))

while new_distance < radius*2:
    print(f'\nUh oh, weird physics at index {index}, distance = {new_distance}\n')
    next_a_position = np.sum([next_a_position, a_velocity*delta/2], axis = 0)
    next_b_position = np.sum([next_b_position, b_velocity*delta/2], axis = 0)
    new_distance = np.sqrt(((next_a_position[0]-next_b_position[0])**2) + ((next_a_position[1]-next_b_position[1])**2))
    print('bumped! new distance is', new_distance)

In [ ]:
%matplotlib inline
a_position = np.array([[5,3]])[0]
b_position = np.array([[3,5]])[0]

result = [a_position[0]+b_position[0], a_position[1]+b_position[1]]

plt.plot([0,a_position[0]],[0,a_position[1]])
plt.plot([0,b_position[0]],[0,b_position[1]])
plt.plot([0,result[0]],[0,result[1]])

In [ ]:
##simple line animation, color changes with height
%matplotlib notebook
fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot([], [], )

cmap=plt.get_cmap('plasma')


def init():
    ax.set_xlim(-0.2, 1.2)
    ax.set_ylim(-0.2, 1.2)
    count = 0
    return ln,

global count
count = 0
def update(frames):
    xdata.append(frames[0])
    ydata.append(frames[1])
    ln.set_data(xdata, ydata)

    ln.set_color(cmap(frames[0]))
    #ax.set_xlim(0, frame+0.2)
    return ln,

ani = FuncAnimation(fig, update, frames=positions,
                    init_func=init, blit=True, interval=5)
plt.show()

In [ ]:
#working model to quickly show trajectory path
x0, y0 = 0,0
v0x, v0y = 5,8
positions = np.array([[x0,y0]])
velocities = np.array([[v0x,v0y]])

elasticity = 0.8

timesteps = np.linspace(0, 4, 1000)
delta = (np.max(timesteps)-np.min(timesteps))/len(timesteps)


def get_next_step(position, velocity, delta):
    drag = 0.1
        
    if position[1] < 0:
        position[1] = 0
        velocity[1] = -elasticity*velocity[1]
    if position[1] > 1:
        position[1] = 1
        velocity[1] = -elasticity*velocity[1]
    if position[0] < 0:
        position[0] = 0
        velocity[0] = -elasticity*velocity[0]
    if position[0] > 1:
        position[0] = 1
        velocity[0] = -elasticity*velocity[0]
    
    next_position = np.sum([position,velocity*delta], axis = 0)


    acceleration_x = -drag*velocity[0]**2
    acceleration_y = (-drag*velocity[1]**2) - 9.81


    next_velocity_x = velocity[0] + acceleration_x*delta
    next_velocity_y = velocity[1] + acceleration_y*delta

    next_velocity = np.array([next_velocity_x, next_velocity_y])
    
    #print(f'Position: {next_position}      Velcocity = {next_velocity}')

    return next_position, next_velocity


for index, step in enumerate(timesteps):
        
    newposition, newvelocity = get_next_step(positions[index], velocities[index], delta)
        
    positions = np.append(positions, [newposition], axis = 0)
    velocities = np.append(velocities, [newvelocity], axis = 0)


%matplotlib inline
plt.plot(positions[:,0], positions[:,1]);

In [ ]:
# very simple math demo/test
%matplotlib notebook
steps = 500
mass = 1         # projectile mass in kg
cd = 0.5         # drag coefficient in 
A = 0.5          # reference area in m^2
v0 = 10      # initial velocity in m/s
init_angle = 45  # initial angle above horizontal

ro = 1.225       # density of air in kg/m^3

time = 2     # time observed in seconds

a = -9.81

t = np.linspace(0, time, steps)

x = v0*t + a*(t**2)/2

v = v0 + a*t

plt.plot(t,x);

In [ ]:
# early animation test
%matplotlib notebook
fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot([], [], 'b')

def init():
    #ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1.2, 1.2)
    return ln,

def update(frame):
    xdata.append(frame)
    ydata.append(np.sin(frame))
    ln.set_data(xdata, ydata)
    ax.set_xlim(0, frame+0.2)
    return ln,

ani = FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128),
                    init_func=init, blit=True)
plt.show()

In [ ]:
 # janky method for math, not in use
steps = 500
mass = 1         # projectile mass in kg
cd = 0.5         # drag coefficient in 
A = 0.5          # reference area in m^2
init_v = 10      # initial velocity in m/s
init_angle = 45  # initial angle above horizontal

ro = 1.225       # density of air in kg/m^3

#initial position, velocitites, and accelerations [x,y]
position = [[0,0]]

velocity = [[init_v*np.cos(init_angle*np.pi/180), init_v*np.sin(init_angle*np.pi/180)]]

acceleration = [[-(0.5*cd*A*ro*init_v_x**2)/mass, -(0.5*cd*A*ro*init_v_y**2)/mass - 9.81]]

timeline = np.linspace(0,10,steps)

for step, time in enumerate(timeline):
    timedelta = (max(timeline)-min(timeline))/len(timeline)
    
    #update acceleration values
    xaccel = -(0.5*cd*A*ro*(velocity[step][0]**2))/mass
    yaccel = -(0.5*cd*A*ro*(velocity[step][1]**2))/mass - 9.81
    
    acceleration = np.append(acceleration, [[xaccel, yaccel]], axis = 0)
    
    #update velocity values
    xvel = velocity[step][0] + timedelta*xaccel
    
    if position[step][1] > 0:
        yvel = velocity[step][1] + timedelta*yaccel
    else:
        yvel = -0.9*(velocity[step][1] + timedelta*yaccel)
    
    velocity = np.append(velocity, [[xvel, yvel]], axis = 0)
    
    #update the position based on current velocity & time passed
    xpos = position[step][0] + timedelta*xvel
    ypos = position[step][1] + timedelta*yvel
    
    position = np.append(position, [[xpos, ypos]], axis = 0)
    
    #print(f'ypos = {np.round(ypos,3)},     yvel = {np.round(yvel,3)},      yaccel = {np.round(yaccel,4)}')

In [ ]:
plt.plot(position[:steps,0], position[:steps,1]);